### Install Dependencies

In [1]:
'''

#Tested with pytorch version 1.6.0 and torch.version.cuda 10.2
!pip install torch==1.6.0

#Install pytorch geometric
import torch
def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')
  
CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)
!pip install -q torch-scatter==latest+{CUDA}     -f https://pytorch-geometric.com/whl/torch-{TORCH}.html
!pip install -q torch-sparse==latest+{CUDA}      -f https://pytorch-geometric.com/whl/torch-{TORCH}.html
!pip install -q torch-cluster==latest+{CUDA}     -f https://pytorch-geometric.com/whl/torch-{TORCH}.html
!pip install -q torch-spline-conv==latest+{CUDA} -f https://pytorch-geometric.com/whl/torch-{TORCH}.html
!pip install -q git+https://github.com/rusty1s/pytorch_geometric.git

# Install the ncapi client
#!pip install -U -q ncapi-client # For bike-demo dataset only


# Install plotly for visualizations
!pip install -q plotly


!pip install meshio
!pip install -q openmesh
!pip install -q tqdm

'''





"\n\n#Tested with pytorch version 1.6.0 and torch.version.cuda 10.2\n!pip install torch==1.6.0\n\n#Install pytorch geometric\nimport torch\ndef format_pytorch_version(version):\n  return version.split('+')[0]\n\nTORCH_version = torch.__version__\nTORCH = format_pytorch_version(TORCH_version)\n\ndef format_cuda_version(version):\n  return 'cu' + version.replace('.', '')\n  \nCUDA_version = torch.version.cuda\nCUDA = format_cuda_version(CUDA_version)\n!pip install -q torch-scatter==latest+{CUDA}     -f https://pytorch-geometric.com/whl/torch-{TORCH}.html\n!pip install -q torch-sparse==latest+{CUDA}      -f https://pytorch-geometric.com/whl/torch-{TORCH}.html\n!pip install -q torch-cluster==latest+{CUDA}     -f https://pytorch-geometric.com/whl/torch-{TORCH}.html\n!pip install -q torch-spline-conv==latest+{CUDA} -f https://pytorch-geometric.com/whl/torch-{TORCH}.html\n!pip install -q git+https://github.com/rusty1s/pytorch_geometric.git\n\n# Install the ncapi client\n#!pip install -U -q nc

In [2]:
#!pip install torch==1.6.0
#!pip install blitz-bayesian-pytorch

### Import Modules

In [12]:
import pickle
import argparse
import os
import os.path as osp
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch_geometric.transforms as T
from psbody.mesh import Mesh
from torch_geometric.data import InMemoryDataset
from torch_geometric.data import Data
from reconstruction import AE, run, eval_error
from datasets import MeshData
from utils import utils, writer, DataLoader, mesh_sampling

parser = argparse.ArgumentParser(description='mesh autoencoder')
parser.add_argument('--exp_name', type=str, default='car_interpolation_exp')
parser.add_argument('--dataset', type=str, default='cars')
parser.add_argument('--split', type=str, default='interpolation')
parser.add_argument('--test_exp', type=str, default='bareteeth')
parser.add_argument('--n_threads', type=int, default=4)
parser.add_argument('--device_idx', type=int, default=0)

# network hyperparameters
parser.add_argument('--out_channels',
                    nargs='+',
                    default=[32, 32, 32, 64],
                    type=int)
parser.add_argument('--latent_channels', type=int, default=16)
parser.add_argument('--in_channels', type=int, default=3)
parser.add_argument('--seq_length', type=int, default=[9, 9, 9, 9], nargs='+')
parser.add_argument('--dilation', type=int, default=[1, 1, 1, 1], nargs='+')

# optimizer hyperparmeters
parser.add_argument('--optimizer', type=str, default='Adam')
parser.add_argument('--lr', type=float, default=1e-4)
parser.add_argument('--lr_decay', type=float, default=0.99)
parser.add_argument('--decay_step', type=int, default=10)
parser.add_argument('--weight_decay', type=float, default=0)

# training hyperparameters
parser.add_argument('--batch_size', type=int, default=1)
parser.add_argument('--epochs', type=int, default=5000)

# others
parser.add_argument('--seed', type=int, default=1)

# args = parser.parse_args()
args = parser.parse_args("")

args.data_fp = osp.join('data', args.dataset)
args.out_dir = osp.join('reconstruction', 'out', args.exp_name)
args.checkpoints_dir = osp.join(args.out_dir, 'checkpoints')
print(args)

utils.makedirs(args.out_dir)
utils.makedirs(args.checkpoints_dir)

writer = writer.Writer(args)
device = torch.device('cuda', args.device_idx)
torch.set_num_threads(args.n_threads)

# deterministic
torch.manual_seed(args.seed)
cudnn.benchmark = False
cudnn.deterministic = True

Namespace(batch_size=1, checkpoints_dir='reconstruction/out/car_interpolation_exp/checkpoints', data_fp='data/cars', dataset='cars', decay_step=10, device_idx=0, dilation=[1, 1, 1, 1], epochs=5000, exp_name='car_interpolation_exp', in_channels=3, latent_channels=16, lr=0.0001, lr_decay=0.99, n_threads=4, optimizer='Adam', out_channels=[32, 32, 32, 64], out_dir='reconstruction/out/car_interpolation_exp', seed=1, seq_length=[9, 9, 9, 9], split='interpolation', test_exp='bareteeth', weight_decay=0)


In [13]:
# template_fp = osp.join('data/car.obj')
# transform_fp = osp.join(args.data_fp, 'transform.pkl')
# if not osp.exists(transform_fp):
#     print('Generating transform matrices...')
#     mesh = Mesh(filename=template_fp)
#     ds_factors = [4, 4, 4, 4]
# #     ds_factors = [2, 2, 2, 2, 2, 2]
#     _, A, D, U, F, V = mesh_sampling.generate_transform_matrices(
#         mesh, ds_factors)
#     tmp = {
#         'vertices': V,
#         'face': F,
#         'adj': A,
#         'down_transform': D,
#         'up_transform': U
#     }

#     with open(transform_fp, 'wb') as fp:
#         pickle.dump(tmp, fp)
#     print('Done!')
#     print('Transform matrices are saved in \'{}\''.format(transform_fp))
# else:
#     with open(transform_fp, 'rb') as f:
#         tmp = pickle.load(f, encoding='latin1')

In [5]:
# spiral_indices_list = [
#     utils.preprocess_spiral(tmp['face'][idx], args.seq_length[idx],
#                             tmp['vertices'][idx],
#                             args.dilation[idx]).to(device)
#     for idx in range(len(tmp['face']) - 1)
# ]
# down_transform_list = [
#     utils.to_sparse(down_transform).to(device)
#     for down_transform in tmp['down_transform']
# ]
# up_transform_list = [
#     utils.to_sparse(up_transform).to(device)
#     for up_transform in tmp['up_transform']
# ]

In [6]:
# mesh = Mesh(filename=template_fp)
# ds_factors = [4, 4, 4, 4]
# #     ds_factors = [2, 2, 2, 2, 2, 2]
# _, A, D, U, F, V = mesh_sampling.generate_transform_matrices(
#     mesh, ds_factors)
# tmp = {
#     'vertices': V,
#     'face': F,
#     'adj': A,
#     'down_transform': D,
#     'up_transform': U
# }
# spiral_indices_list = [
#     utils.preprocess_spiral(tmp['face'][idx], args.seq_length[idx],
#                             tmp['vertices'][idx],
#                             args.dilation[idx]).to(device)
#     for idx in range(len(tmp['face']) - 1)
# ]
# down_transform_list = [
#     utils.to_sparse(down_transform).to(device)
#     for down_transform in tmp['down_transform']
# ]
# up_transform_list = [
#     utils.to_sparse(up_transform).to(device)
#     for up_transform in tmp['up_transform']
# ]

In [7]:
# for x in down_transform_list:
#     print (np.shape(x))
    
    
# for x in up_transform_list:
#     print (np.shape(x))
    
# for x in spiral_indices_list:
#     print (np.shape(x))

In [8]:
import meshio
import torch
import numpy as np
def from_meshio(mesh_list, mesh_obj_list, drag_list):
    r"""Converts a :.msh file to a
    :class:`torch_geometric.data.Data` instance.

    Args:
        mesh (meshio.read): A :obj:`meshio` mesh.
    """
    print("processing geometries...")
    if meshio is None:
        raise ImportError('Package `meshio` could not be found.')

    data_list = []
    #for mesh in mesh_list:
    for i in range (len(mesh_list)):
        print(i)
        try:

            mesh_io = meshio.read(mesh_list[i])
            #print(mesh_list[i])

            
            pos = np.expand_dims(mesh_io.points.astype(np.float32), axis=0)
            
            pos = torch.from_numpy(pos).to(torch.float)
            
            face = torch.from_numpy(mesh_io.cells[0].data.astype(np.float32)).to(torch.long).t().contiguous()
#             f2e = T.FaceToEdge(remove_faces=False)
#             edge_index = f2
            #print(face.shape)

            f = mesh_io.cells[0].data#.astype(np.float32)
            #print(f.shape[1])
            
            print(mesh_list[i])
            print(mesh_obj_list[i])
            mesh = Mesh(filename=mesh_obj_list[i])
            ds_factors = [4, 4, 4, 4]
            #     ds_factors = [2, 2, 2, 2, 2, 2]
            _, A, D, U, F, V = mesh_sampling.generate_transform_matrices(
                mesh, ds_factors)
            tmp = {
                'vertices': V,
                'face': F,
                'adj': A,
                'down_transform': D,
                'up_transform': U
            }
            print(tmp)
            spiral_indices_list = [
                utils.preprocess_spiral(tmp['face'][idx], args.seq_length[idx],
                                        tmp['vertices'][idx],
                                        args.dilation[idx])#.contiguous()#.to(device)
                for idx in range(len(tmp['face']) - 1)
            ]
            print(spiral_indices_list)
        
#             spiral = preprocess_spiral(f, args.seq_length)

            #print(spiral)


#             indices = spiral.to(torch.long).contiguous()


            #edge_index = torch.cat([tetra[:2], tetra[1:3, :], tetra[-2:], tetra[::2], tetra[::3], tetra[1::2]], dim=1)
            #print(np.shape(mesh.point_data['point_scalars']))
            #y_node = torch.from_numpy(mesh.point_data['point_scalars'][:,0])
            y_node = torch.from_numpy(mesh_io.point_data['p'].astype(np.float32))
            y_stress = torch.from_numpy(mesh_io.point_data['wallShearStress'].astype(np.float32))




            '''f = open(drag_list[i], 'r')
            cd = f.read()
            cd = np.asarray(float(cd))
            cd = torch.from_numpy(cd).to(torch.float)
            f.close'''


            data = Data(x=pos, face=face, y_node = y_node, y_stress = y_stress, spiral_indices_list=spiral_indices_list, down_transform_list= tmp['down_transform'], up_transform_list = tmp['up_transform'])

            data_list.append(data)
        except Exception as e:
            print('Error in ', mesh_list[i], e)

    return data_list#Data(x=pos, tetra=tetra, press_y = press_y)#, edge_index = edge_index)

#dataset = from_meshio(mesh_list[:10], drag_list)

def to_meshio(data):
    r"""Converts a :class:`torch_geometric.data.Data` instance to a
    :obj:`.msh format`.

    Args:
        data (torch_geometric.data.Data): The data object.
    """

    if meshio is None:
        raise ImportError('Package `meshio` could not be found.')
    
    points = data.x.detach().cpu().numpy()
    tetra = data.tetra.detach().t().cpu().numpy()
    point_scalars = data.y_node.detach().numpy()
    
    cells = [("tetra", tetra)]
    point_data = [("point_data", point_scalars)]

    return meshio.Mesh(points, cells,point_data) 

In [9]:
import os
from torch_geometric.data import DataLoader
import random
#mesh = meshio.read('meshData/0/quadpress_smpl.vtk')
#mesh = meshio.read('meshtri.vtk')
#meshio.write('test.stl', mesh)



print("traversing root directory to find geometries...")
path = '/media/asad/Work/Documents/Emulator/AutoDesk_GP/mlcfd_data/mlcfd_data/training_data/'
path = '/media/asad/Work/Documents/Emulator/AustalDataset/extracted/'
pathobj = '/media/asad/Work/Documents/Emulator/AustalDataset/shipsObj/'
keyword_p = 'quadpress_smpl'
keyword_d = 'cd.txt'

keyword_p = 'vessel.vtk'
keyword_d = 'drag_coeffs.json'
mesh_list = []
drag_list = []
mesh_list_obj = []
for root, dirs, files in os.walk(path):
    path = root.split(os.sep)
    #print((len(path) - 1) * '---', os.path.basename(root))
    for file in files:
        
        if keyword_p in file:
            mesh_list.append(root+'/'+file)
            #print((root)+file, " has the keyword p")
        if keyword_d in file:
            drag_list.append(root+'/'+file)
#             print((root).split('/')[-1])#+file, " has the keyword d")
            case = (root).split('/')[-1]
            mesh_list_obj.append(pathobj+case+'/'+'vessel.obj')
        

dataset = from_meshio(mesh_list[:5], mesh_list_obj[:5], drag_list[:5])

traversing root directory to find geometries...
processing geometries...
0
/media/asad/Work/Documents/Emulator/AustalDataset/extracted/case653/vessel.vtk
/media/asad/Work/Documents/Emulator/AustalDataset/shipsObj/case653/vessel.obj
calculating factors
factors done
vert connectivity
vert connectivity done
decimator...
decimator done!
vert connectivity factors
decimator...
decimator done!
vert connectivity factors
decimator...
decimator done!
vert connectivity factors
decimator...
decimator done!
vert connectivity factors
{'vertices': [array([[-3.43361832e-02, -5.42159527e-18,  7.61456394e+00],
       [-3.43361832e-02,  5.42159527e-18,  7.61456394e+00],
       [-1.83069259e-02, -2.50297695e-01,  7.63137341e+00],
       ...,
       [ 7.97443085e+01,  8.70944595e+00, -9.16492001e-33],
       [ 7.98957138e+01,  8.62114334e+00, -9.16492001e-33],
       [ 8.00000992e+01,  8.56025982e+00, -9.16492001e-33]]), array([[-3.43361832e-02, -5.42159527e-18,  7.61456394e+00],
       [-3.43361832e-02,  

[tensor([[    0,  3276,  3273,  ...,  3667,  3275,  3271],
        [    1,     9,     2,  ...,     4,    14,    26],
        [    2,  3270,  3274,  ...,     4,    14,    26],
        ...,
        [78069, 78070, 78068,  ..., 78066, 78063, 78061],
        [78070, 78071, 78068,  ..., 78067, 78063, 78061],
        [78071, 78068, 78070,  ..., 78063, 78061, 78057]]), tensor([[    0,    82,     3,  ...,    84,    80,    27],
        [    1,     9,     2,  ...,    14,    26,    79],
        [    2,    79,    83,  ...,     4,    14,    26],
        ...,
        [19515, 19516, 19160,  ..., 18927, 19396, 19517],
        [19516, 18927, 19396,  ..., 18948, 18608, 18455],
        [19517, 19472, 19304,  ..., 18389, 19469, 19471]]), tensor([[   0,    5,    2,  ...,   20,    6,   13],
        [   1,   21,   19,  ...,    4,   41,   62],
        [   2,    8,    0,  ...,   11,   18,   20],
        ...,
        [4877, 4578, 4760,  ..., 4580, 4876, 4861],
        [4878, 4748, 4879,  ..., 4754, 4445, 4449],


[tensor([[    0,    53,    52,  ...,   124,    54,  3586],
        [    1,    55,   123,  ...,  1622,     2,    57],
        [    2,    55,  1622,  ...,   123,     1,  2201],
        ...,
        [82140, 82133, 82132,  ..., 82137, 82134, 82130],
        [82141, 82142, 82139,  ..., 82133, 82132, 82136],
        [82142, 82139, 82141,  ..., 82138, 82134, 82130]]), tensor([[    0,     7,    19,  ...,   135,   198,    85],
        [    1,    22,     2,  ...,    47,    67,    29],
        [    2,     0,    19,  ...,    59,    30,     1],
        ...,
        [20533, 20531, 20532,  ..., 20448, 13989, 20521],
        [20534, 20533, 20396,  ..., 20531, 20532, 20402],
        [20535, 20450, 20448,  ..., 20378, 20322, 20320]]), tensor([[   0,    4,    3,  ...,   28,   14,   12],
        [   1,    0,    3,  ..., 2325,   20,   28],
        [   2,   23,   13,  ...,    5,   27,   25],
        ...,
        [5131, 5115, 5113,  ..., 5097, 5091, 5093],
        [5132, 5131, 5117,  ..., 2327, 5115, 5113],


[tensor([[    0,   169,   710,  ...,   712,  2417,  2416],
        [    1,    38,    37,  ...,     7,  2668,  2666],
        [    2,     7,    38,  ...,    39,  1221,   295],
        ...,
        [72747, 72744, 72740,  ..., 72742, 72737, 72730],
        [72748, 72749, 72747,  ..., 72740, 72738, 72743],
        [72749, 72747, 72748,  ..., 72745, 72742, 72737]]), tensor([[    0,     4,    21,  ...,    24,    28,    36],
        [    1,   375,    80,  ...,    21,    12,  1070],
        [    2,     7,    22,  ...,    47,   162,   376],
        ...,
        [18185, 18079, 18186,  ..., 17932, 18067, 18187],
        [18186, 16053, 18184,  ..., 18187, 18105, 18103],
        [18187, 18105, 16053,  ..., 18097, 18103, 18101]]), tensor([[   0,    6,    1,  ...,    8,   56,   49],
        [   1,   56,   49,  ...,    0,    6,    8],
        [   2,    5,    7,  ...,   14,   21,   50],
        ...,
        [4544, 4448, 4543,  ..., 4449, 4459, 4451],
        [4545, 4527, 4525,  ..., 4505, 4503, 4509],


[tensor([[    0,  4714,   384,  ...,  4702,  4736,  5066],
        [    1,  4708,  4704,  ...,  1849,  4740,  5058],
        [    2,  4830,  4844,  ...,  2964,  1424,     3],
        ...,
        [80430, 80429, 80426,  ..., 80420, 80422, 80427],
        [80431, 80432, 80426,  ..., 80428, 80425, 80420],
        [80432, 80430, 80426,  ..., 80425, 80420, 80422]]), tensor([[    0,  1753,  1024,  ...,    58,    14,    10],
        [    1,    57,    47,  ...,    11,   113,   817],
        [    2,    92,   482,  ...,  2003,    65,     7],
        ...,
        [20106, 20076, 20097,  ..., 20066, 20088, 20091],
        [20107, 20102, 20100,  ..., 20092, 20094, 20096],
        [20108, 20106, 20101,  ..., 20097, 20095, 20093]]), tensor([[   0,  766,  240,  ..., 1453, 1033,  765],
        [   1,   92,   98,  ...,  118,  609,  902],
        [   2,   37,   55,  ...,   10,   21,   63],
        ...,
        [5025, 5021, 5018,  ..., 5012, 5010, 4988],
        [5026, 4999, 5019,  ..., 4989, 5011, 5013],


[tensor([[    0,  3270,  3266,  ...,   460,  3272,  3368],
        [    1,   478,  1612,  ...,   619,   106,    17],
        [    2,  3469,   363,  ...,  3758,  3759,  3470],
        ...,
        [57685, 57686, 57626,  ..., 57679, 57678, 57676],
        [57686, 57687, 57626,  ..., 57682, 57679, 57678],
        [57687, 57216, 57626,  ..., 57237, 57234, 57231]]), tensor([[    0,    87,   539,  ...,    89,    20,    10],
        [    1,    56,    88,  ...,     7,    11,   670],
        [    2,     3,    56,  ...,     7,    21,     9],
        ...,
        [14419, 14417, 14411,  ..., 14405, 14397, 14409],
        [14420, 14419, 14415,  ..., 14409, 14359, 14412],
        [14421, 14403, 14414,  ..., 14410, 14416, 14302]]), tensor([[   0,   12,    4,  ...,   28,   14,   52],
        [   1,    3,    5,  ...,   95,   39,   21],
        [   2,    6,    0,  ...,   16,   18,   20],
        ...,
        [3603, 3605, 3601,  ..., 3539, 3527, 3483],
        [3604, 3602, 3600,  ..., 3592, 3596, 3599],


In [11]:
print(mesh_list_obj[0])


train_dataset = dataset[:1]
test_dataset = dataset[1:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')


train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

d = train_dataset[0]
print(d)

/media/asad/Work/Documents/Emulator/AustalDataset/shipsObj/case653/vessel.obj
Number of training graphs: 0
Number of test graphs: 0


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
# d = train_dataset[0]
# print(d)

# for x in d.down_transform_list:
#     print(np.shape(x))
# down_transform = [utils.to_sparse(down_transform) for down_transform in d.down_transform_list ]



# for x in down_transform:
#     print(np.shape(x))
    
    
for i, data in enumerate(train_loader):
    for down_transform in data.down_transform_list[0]:
        print((down_transform))
    
#     down_transform = [utils.to_sparse(down_transform) for down_transform in data.down_transform_list[0]]

#     up_transform = [ utils.to_sparse(up_transform)	for up_transform in data.up_transform_list[0]]
    
#     for x in down_transform:
#         print(np.shape(x))
        
#     for x in up_transform:
#         print(np.shape(x))
    
    
#     print((tmp[0]['down_transform']))
#     down_transform_list = [
#     utils.to_sparse(down_transform).to(device)
#     for down_transform in tmp['down_transform']
#     ]
#     up_transform_list = [
#         utils.to_sparse(up_transform).to(device)
#         for up_transform in tmp['up_transform']
#     ]
    
#     print(np.shape(down_transform_list))
    
#     down_transform = [utils.to_sparse(down_transform) for down_transform in down_transform_list ]

#     up_transform = [ utils.to_sparse(up_transform)	for up_transform in up_transform_list]


# # torch.save(dataset, 'tempdata_x50.pt')

# # dataset = torch.load('tempdata_x50.pt')
# random.seed(1234)
# random.shuffle(dataset)

# #dataset = random.Random(12345).shuffle(dataset)

# frac = len(dataset)*0.9

# train_dataset = dataset[:712]
# test_dataset = dataset[712:]
# print(f'Number of training graphs: {len(train_dataset)}')
# print(f'Number of test graphs: {len(test_dataset)}')


# train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# d = train_dataset[1]
# print(d)

In [ ]:
model = AE(args.in_channels, args.out_channels, args.latent_channels,
           spiral_indices_list, down_transform_list,
           up_transform_list, args.seq_length).to(device)

# for data in train_loader:
#     print(data)

# print(np.shape(d.x.view(-1, 3)))

# num_train_graph = len(train_loader)
# num_test_graph = len(test_loader)

# mean = train_loader.data.x.view(num_train_graph, -1, 3).mean(dim=0)
# std = train_dataset.x.view(num_train_graph, -1, 3).std(dim=0)
    
# print('Normalizing...')
# train_dataset.x = (
#     (train_dataset.x.view(num_train_graph, -1, 3) -
#      mean) / std).view(-1, 3)
# test_dataset.x = (
#     (test_dataset.x.view(num_test_graph, -1, 3) -
#      mean) / std).view(-1, 3)
# print('Done!')

In [ ]:

print('Number of parameters: {}'.format(utils.count_parameters(model)))
print(model)

optimizer = torch.optim.Adam(model.parameters(),
                             lr=args.lr,
                             weight_decay=args.weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                            args.decay_step,
                                            gamma=args.lr_decay)


In [ ]:
run(model, train_loader, test_loader, args.epochs, optimizer, scheduler,
    writer, device)
# eval_error(model, test_loader, device, meshdata, args.out_dir)

In [ ]:
# torch.save(model, 'cars_AE_2.pt')

model = torch.load('cars_AE.pt')

### Dataset Creation and Visualisation



In [ ]:
print(d.num_features)
#model = Net(d.num_features, d.num_nodes, spiral_indices).to(device)
# model = Net(3, 1, spiral_indices).to(device)
model = Net(d.num_features, num_classes=1, spiral_length = args.seq_length).to(device)

In [ ]:
optimizer = optim.Adam(model.parameters(),
                       lr=args.lr,
                       weight_decay=args.weight_decay)
scheduler = optim.lr_scheduler.StepLR(optimizer,
                                      args.decay_step,
                                      gamma=args.lr_decay)

In [ ]:
Train = True
if Train:
    run(model, train_loader, test_loader, target, d.num_nodes, args.epochs, optimizer, scheduler, device)
#torch.save(model.state_dict(), 'SpiralNet_car.pt')

### Load and Visualise Dataset

In [ ]:
if Train:
    print("training...")
    torch.save(model.state_dict(), 'SpiralNe_AE_car_5000.pt')
else:
    model.load_state_dict(torch.load('SpiralNet_bayesian_car_press_drag_no_drop_onefc.pt'))
    print("model loaded...")
    
#torch.save(model.state_dict(), 'SpiralNet_bayesian_car_press_drag_no_drop_onefc.pt')

In [ ]:
import torch.nn.functional as F
@torch.no_grad()
def test_eval(model, loader):
    model.eval()
    total_loss = 0
    total_correct = 0
    for data in loader:
        data = data.to(device)
        #out_node, out_graph = model(data.x, data.batch)
        out_node = model(data.x)
        #print(out_graph)
#         loss = F.mse_loss(out_node[0,:,0], data.y_node.float())
        loss = F.l1_loss(out_node[0,:,0], data.y_node.float(), reduction='mean')
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(loader.dataset)

error_train = test_eval(model, train_loader)
print("MSE_train_p: ", error_train) #total number of nodes vertices in a graph

error_test = test_eval(model, test_loader)
print("MSE_test_p: ", error_test)

# @torch.no_grad()
# def test_eval(model, loader):
#     model.eval()
#     total_loss = 0
#     total_correct = 0
#     for data in loader:
#         data = data.to(device)
#         out_node, out_graph = model(data)
#         #print(out_graph)
#         loss = F.mse_loss(out_graph[0], data.cd.float())
#         total_loss += loss.item() * data.num_graphs
#     return total_loss / len(loader.dataset)

# error_train = test_eval(model, train_loader)
# print("MSE_train_drag: ", error_train)#/(3682)) #total number of nodes vertices in a graph

# error_test = test_eval(model, test_loader)
# print("MSE_test_drag: ", error_test)#/(3682))

#MSE_train_p:  0.02416176421675669
#MSE_test_p:  0.028340919857371485
#MSE_train_drag:  3.262181123596277e-05
#MSE_test_drag:  8.782161437001768e-05


# Running predictions: 89it [00:14, 12.58it/s][01-02-21-03:18:55] ncs.scripts.predict | INFO: {
#     "r2/mesh.fields.point_scalars": 0.990871012210846,
#     "r2/scalars.s_0": 0.37898457050323486,
#     "Mean/loss_l1/total": 2.154623031616211,
#     "Mean/loss_l1/mesh.fields.point_scalars": 2.1284544467926025,
#     "Mean/loss_l1/scalars.s_0": 0.026168810203671455
# }
# [01-02-21-03:18:55] ncs.scripts.predict | INFO: Running prediction on /home/container_user/workspace/data/conversion/carconversion2/carconversion2/train
# Running predictions: 89it [00:30, 12.58it/s] [01-02-21-03:20:20] ncs.scripts.predict | INFO: {
#     "r2/mesh.fields.point_scalars": 0.9980176091194153,
#     "r2/scalars.s_0": 0.8438485264778137,
#     "Mean/loss_l1/total": 1.002699851989746,
#     "Mean/loss_l1/mesh.fields.point_scalars": 0.9904414415359497,
#     "Mean/loss_l1/scalars.s_0": 0.012257951311767101
# }

In [ ]:
from sklearn.metrics import r2_score

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)



   
@torch.no_grad()

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

@torch.no_grad()
def evaluate_regression(model,
                        X,
                        y,
                        samples = 100):
    
    y = y.cd.float()
    
    y_pred = [model(X)[1] for i in range(samples)]
    
    return y, y_pred


def test_eval(model, loader):
    model.eval()
    total_loss = 0
    total_correct = 0
    
    y_pred_list= []
    y_list = []
    for data in loader:
        data = data.to(device)
        y, y_pred = evaluate_regression(model,data,data,samples=25)
        y_list.append(y)
        y_pred_list.append(y_pred)
    return y_list, y_pred_list


def plot_uncertainity(y_list, y_pred_list):
    size = np.shape(y_list)[0]

    y_mean = np.mean(np.asarray(y_pred_list).astype(np.float32), axis=1)
    y_sigma = np.std(np.asarray(y_pred_list).astype(np.float32), axis=1)

    test_labels= np.array(y_list).astype(np.float32).reshape((size))
    testPredict = np.array(y_mean).astype(np.float32).reshape((size))
    
    MAPE_error = mean_absolute_percentage_error(test_labels, testPredict)


    from matplotlib.pyplot import figure
    figure(num=None, figsize=(14, 12), dpi=80, facecolor='w', edgecolor='k')
    

    plt.scatter(test_labels, testPredict, color='b', label='Predictive mean')

    plt.plot(test_labels, test_labels, '-r', label='Training data')

    plt.errorbar(test_labels, testPredict, yerr=y_sigma, color='black', fmt='+', label='Epistemic uncertainty')

    plt.title('Prediction')
    plt.legend()
    plt.show()
    error = test_labels - testPredict
    
    plt.hist(error, density=False, bins=100)  # `density=False` would make counts
    plt.title('Distribution of the magnitude of errors')
    plt.ylabel('No. of Samples')
    plt.xlabel('Errors in cd')
    plt.show()
    print("Mean of error: ", np.mean(error))
    print("Standard Deviation of error: ", np.std(error))
    print("Mean absolute percentage error: ", MAPE_error)
    print("R2 Score: ", r2_score(test_labels , testPredict))
    
    
    '''plt.hist(error/np.std(error), density=False, bins=100)  # `density=False` would make counts
    plt.title('Errors normalized by Standard Deviation')
    plt.ylabel('No. of Samples')
    plt.xlabel('Errors in cd')
    plt.show()
    print("Mean: ", np.mean(error/np.std(error)))
    print("Standard Deviation", np.std(error)/np.std(error))  '''

    
 
    
##########################################################################################



y_list, y_pred_list = test_eval(model, train_loader)

print("Train dataset plots")

plot_uncertainity(y_list, y_pred_list)

##########################################################################################


y_list, y_pred_list = test_eval(model, test_loader)

print("\n\n\nTest dataset plots")

plot_uncertainity(y_list, y_pred_list)




In [ ]:
import plotly.offline as py
import plotly.graph_objs as go
def visualize_sample(sample, predictions=None):
    verts = np.array(sample.x)
    faces = np.array(sample.tetra)
    #preds = predictions["output_fields"][0, ...] if predictions else np.array(sample["output_fields"])
    preds = np.array(sample.y_node)
    preds1 = preds.reshape((3682,1))
    #print('verts: ', np.shape(verts), 'face: ', np.shape(faces), 'preds: ', np.shape(preds))
    faces1 = np.swapaxes(faces,0,1)
    print(np.shape(preds1), preds1)
    points =verts
    cells = [("quad", faces1)]
    meshio.write_points_cells(
        "original.vtk",
        points,
        cells,
        point_data={'point_scalars': preds1},

    )

    x, y, z = verts[:,0], verts[:,1], verts[:,2]
    I, J, K, L = faces[0,:], faces[1,:], faces[2,:], faces[3,:]
    #preds = np.clip(preds, -0.5, 0.5)
    #print(np.shape(preds))
  
    preds = (preds-np.min(preds))/(np.max(preds)-np.min(preds)) #normalize
    
    trace = go.Mesh3d(x=x,y=y,z=z,i=I,j=J,k=K,intensity=preds, showscale=True, cmax=1,cmin=0, colorbar_title='press_label')
     
    return preds, go.Figure(data=[trace], layout=go.Layout(scene=dict(aspectmode="data")))
    
    
    
def visualize_preds(sample, preds, pp):
    #sample = sample
    verts = np.array(sample.x)
    faces = np.array(sample.tetra)
    x, y, z = verts[:,0], verts[:,1], verts[:,2]
    I, J, K = faces[0,:], faces[1,:], faces[2,:]
    #preds = np.clip(preds, -0.5, 0.5)
    #print(np.shape(preds))
    
    preds1 = pp.reshape((3682,1))
    #print('verts: ', np.shape(verts), 'face: ', np.shape(faces), 'preds: ', np.shape(preds))
    faces1 = np.swapaxes(faces,0,1)
    print(np.shape(preds1), preds1)
    points =verts
    cells = [("quad", faces1)]
    meshio.write_points_cells(
        "diff.vtk",
        points,
        cells,
        point_data={'point_scalars': preds1},

    )
    
    trace = go.Mesh3d(x=x,y=y,z=z,i=I,j=J,k=K,intensity=preds,showscale=True, cmax=1,cmin=0, colorbar_title='press')
    
    return preds, go.Figure(data=[trace], layout=go.Layout(scene=dict(aspectmode="data")))

In [ ]:
# train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
import pylab as plot
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

params = {'legend.fontsize': 20,
          'legend.handlelength': 2, 'xtick.labelsize':20, 'ytick.labelsize':20,}
plot.rcParams.update(params)
@torch.no_grad()
def vis_predictions(model, loader, idx):

  for i, data_i in enumerate(loader):
    #print(i)
    if i==idx:
        label, vis_y = visualize_sample(data_i)
        
#         vis_y.show()
        
        
        
        
        model.eval()
        data_i = data_i.to(device)
        
    
#         print(np.shape(data_i.x[0,:,:]))
        
        out_node = model(data_i.x)
        figure(num=None, figsize=(12, 8), dpi=80, facecolor='w', edgecolor='k')

        plt.plot(data_i.y_node.to('cpu'))
        plt.plot(out_node[0,:,0].to('cpu'))
        
        plt.show()
        preds = out_node.detach().to('cpu').numpy()
        pp = preds
        print(pp)
        x = preds[:,0]
        normalized_pred = (x-x.min())/(x.max()-x.min())
        prediction, vis_pred = visualize_preds(data_i.to('cpu'),normalized_pred, pp[:,0])
        #print('prediction drag: ', out_graph[0].to('cpu').numpy())
        vis_pred.show()
        prediction = prediction#.detach().to('cpu')
        x = np.fabs(normalized_pred - label)
        
        #normalized = (x-x.min())/(x.max()-x.min())

        _, diff = visualize_preds(data_i.to('cpu'), x , x)
        print('Difference')
        diff.show()
        
        break

        
#############################################################

sample = 25
        
vis_predictions(model, train_loader, sample)



In [ ]:
'''def to_meshio(data):
    r"""Converts a :class:`torch_geometric.data.Data` instance to a
    :obj:`.msh format`.

    Args:
        data (torch_geometric.data.Data): The data object.
    """

    if meshio is None:
        raise ImportError('Package `meshio` could not be found.')
    
    points = data.x.detach().cpu().numpy()
    tetra = data.tetra.detach().t().cpu().numpy()
    point_scalars = data.y_node.detach().numpy().reshape(3682,1)
    print(np.shape(point_scalars))
    #point_data=point_data
    cells = [("tetra", tetra)]
    point_data = {"point_scalars": point_scalars}

    return meshio.Mesh(points, cells, point_data)#,point_data) 

# mesh points
import pyvista as pv
for i, data_i in enumerate(test_loader):
    
    mesh = to_meshio(data_i)
    print(mesh)
    mesh.write("foo.vtk")
    '''verts = np.array(data_i.x)
    faces = np.swapaxes(np.array(data_i.tetra),0,1)
    #preds = predictions["output_fields"][0, ...] if predictions else np.array(sample["output_fields"])
    preds = np.array(data_i.y_node)
    
    print(np.shape(verts), np.shape(faces))

    vertices = np.array([[0, 0, 0],
                     [1, 0, 0],
                     [1, 1, 0],
                     [0, 1, 0],
                     [0.5, 0.5, -1]])

    # mesh faces
    faces = np.hstack([[4, 0, 1, 2, 3],  # square
                   [3, 0, 1, 4],     # triangle
                   [3, 1, 2, 4]])    # triangle
    
    print(np.shape(vertices), np.shape(faces))
    
    #surf = pv.PolyData(verts, faces)
    

    # plot each face with a different color
    #surf.plot()#(scalars=np.arange(3), cpos=[-1, 1, 0.5])'''
    break;
mesh = pv.read('foo.vtk')
'''p = pv.Plotter()
p.add_mesh(mesh)
p.show(notebook=False,auto_close=False, use_panel=True)'''

#!jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-matplotlib jupyterlab-datawidgets itkwidgets

#!pip install itkwidgets
from itkwidgets import view

view(geometries=mesh, geometry_colors='w')

'''